# eCREST_notebook

Basic functions for reconstructing cells.

# Setup

Do the following two setup steps regardless of how you will be using this script. 

## 1. Imports

Run the following code cell to import the necessary packages and modules. 

In [1]:
############################################################################################################################ 
# Get the latest CREST files for each ID within the target folder (dirname)

from pathlib import Path
import json
from sqlite3 import connect as sqlite3_connect
from sqlite3 import DatabaseError
from igraph import Graph as ig_Graph
from igraph import plot as ig_plot
from scipy.spatial.distance import cdist
from random import choice as random_choice
from itertools import combinations
from numpy import array, unravel_index, argmin, mean,unique,nan
import pandas as pd
from copy import deepcopy
from datetime import datetime
from time import time
import neuroglancer
from webbrowser import open as wb_open
from webbrowser import open_new as wb_open_new
import neuroglancer

# from eCREST_cli_beta import ecrest, import_settings
from eCREST_cli import ecrest, import_settings, get_cell_filepaths

The 'ecrest' class has been imported from eCREST_cli.py

An instance of this object will be able to do things like:
- open an neuroglancer viewer for proofrieading (see "Proofread using CREST")
    - add-remove segments (using graph feature for efficiency)
    - format itself and save itself as a CREST-style .json
- add or remove annotation layers (see "Annotation Layers")
- check for overlap with other .json files in a directory folder (see "check for overlap")
- label cell structures
- add base_segments from a list (see "add segments")
- import annotations from another file (see "Annotation Import")
- convert from neuroglancer json (see "Convert From Neuroglancer to eCREST")
    - format itself and save itself as a CREST-style .json
    


## 2. Settings definitions

Whether you are converting from neuroglancer or creating a new reconstruction, the settings_dict parameters is needed to create CREST json files with correct formatting. 
- 'save_dir' : the directory where JSON files are saved 
- 'cred' and 'db_path' : specify the path to the agglomeration database file on your local computer. 

In [2]:
path_to_settings_json = '/Users/kperks/Documents/ell-connectome/eCREST-local-files/settings_dict.json'
settings_dict = import_settings(path_to_settings_json)

# Proofread using (e)CREST

The ```ecrest``` class defined in eCREST_cli.py can be used to proofread base_segment reconstructions enhanced by the agglomeration database.

An instance of this class can be initialized with either:
- ecrest(segment_id): a "main_base_id" in *int* format
- ecrest(filepath): an existing CREST .json file
- ecrest(segment_id, segment_list): the main_base_id from the neuroglancer file you are converting and a list of base_segments.

The ```launch_viewer``` flag default is "False" so that you can interact with the contents of a reconstruction without actually opening it visually in a neuroglancer tab. **NOTE**: Some ecrest functions require that the ecrest instance is created with ```launch_viewer==True```.

In [3]:
todo_f = Path(settings_dict['save_dir']) / 'todo_presynaptic/lg_301787806' #lf_393325331' #

In [4]:
len(get_cell_filepaths(todo_f))

100

## NEW reconstruction from segment ID

If you wanted to start reconstructing a new cell from a main base segment, you would use the following code block to launch.

To change the save location you can specify the ```directory_path``` flag in the ```save_cell_graph()``` module

In [ ]:
segment_id = 475845127
crest = ecrest(settings_dict,segment_id = segment_id, launch_viewer=True)

crest.change_key_binding({"alt+mousedown0" : "add-or-remove-seg"})

In [34]:
crest.add_endpoint_annotation_layers(['soma'],link=True) # spine_inputs
print(crest.get_ctype('manual'))

[]


In [35]:
print('overlap in main network:'); df = crest.check_duplicates(base_segments_net); display(df)
print('overlap in to do folder:'); df = crest.check_duplicates(base_segments_todo); display(df)

overlap in main network:


,self,dups,overlap-percent,number_seg_lap


overlap in to do folder:


,self,dups,overlap-percent,number_seg_lap


1 other base segments in the agglo segment; max number can add is 1000
1 clusters of connected components. Connecting these clusters with nearest base segments.
Added 1 base segments from agglomerated segment 482713240, linked base segments 482713444 and 482713240, 915nm apart, 
1 other base segments in the agglo segment; max number can add is 1000
1 clusters of connected components. Connecting these clusters with nearest base segments.
Added 1 base segments from agglomerated segment 482713478, linked base segments 482713511 and 482713478, 462nm apart, 
1 other base segments in the agglo segment; max number can add is 1000
1 clusters of connected components. Connecting these clusters with nearest base segments.
Added 1 base segments from agglomerated segment 482697830, linked base segments 482697806 and 482697830, 569nm apart, 
1 other base segments in the agglo segment; max number can add is 1000
1 clusters of connected components. Connecting these clusters with nearest base segments.

In [30]:
cell_type = 'grc-d' # Assign the cell type then run the code cell

crest.define_ctype(cell_type,"manual")

In [31]:
crest.save_cell_graph()

Saved cell 481521098 reconstruction locally at 2024-01-19 10.05.29



## EDIT reconstruction from file

If you wanted to edit a reconstruction from an existing file, you would use the following code block to launch.

Specify the cell_id and the path to the directory that cell is in. 

> NOTE: You can also directly copy paste the full filepath to the cell you want to open and pass it to the ```filepath``` flag.  
In that case, the only code you need is crest = ecrest(settings_dict,filepath= [*paste filepath here*], launch_viewer=True)

To change the save location you can specify the ```directory_path``` flag in the ```save_cell_graph()``` module

> To overwrite a file (not recommended), specify ```(directory_path = cell_filepaths[cell_id].parent, file_name = cell_filepaths[cell_id].name)```

In [ ]:
'UK' todo
[
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '381969662',
 '638482210',
 '294966040',
 '469962237',
 '297224631',
 '297224094',
 '122129803',
 '297223669',
 '211365366',
 '211364877',
 '124327712',
 '211364285',
 '211347145',
 '124330030',
 '210218055',
 '210216901',
 '211361897',
 '124345289']

In [11]:
directory_path = Path(settings_dict['save_dir']) #/ 'todo_presynaptic/mg1_299496636' #/ 'kp/pfs'#/ 'volume-subsample-all/in_progress'  ## specify the directory path
cell_filepaths = get_cell_filepaths(directory_path) # gets filepaths for all cells in a directory
# cell_filepaths = cell_filepaths_mainnet

In [12]:
cell_id = '480314495' # specify the cell id

crest = ecrest(settings_dict,filepath= cell_filepaths[cell_id], launch_viewer=True)

crest.change_key_binding({"alt+mousedown0" : "add-or-remove-seg"})

updating viewer status message: Current Base Segment Counts: unknown: 44, axon: 0, basal dendrite: 0, apical dendrite: 0, dendrite: 0, multiple: 0


Map({"dblclick0": "add-or-remove-seg", "alt+mousedown2": "mark-branch-in-colour", "shift+mousedown2": "change-anchor-seg", "alt+mousedown0": "add-or-remove-seg"})

2 other base segments in the agglo segment; max number can add is 1000
1 clusters of connected components. Connecting these clusters with nearest base segments.
Added 2 base segments from agglomerated segment 133488992, linked base segments 133488699 and 133489033, 3136nm apart, 
1 other base segments in the agglo segment; max number can add is 1000
1 clusters of connected components. Connecting these clusters with nearest base segments.
Added 1 base segments from agglomerated segment 132341037, linked base segments 132341979 and 132341037, 2900nm apart, 
1 other base segments in the agglo segment; max number can add is 1000
1 clusters of connected components. Connecting these clusters with nearest base segments.
Added 1 base segments from agglomerated segment 46470461, linked base segments 132341037 and 46470461, 3095nm apart, 
1 other base segments in the agglo segment; max number can add is 1000
1 clusters of connected components. Connecting these clusters with nearest base segments

In [14]:
cell_type = 'fov' # Assign the cell type then run the code cell

crest.define_ctype(cell_type,"manual")

In [15]:
crest.save_cell_graph(directory_path = cell_filepaths[cell_id].parent, file_name = cell_filepaths[cell_id].name)

Saved cell 480314495 reconstruction locally at 2024-01-19 09.41.57


In [ ]:
# crest.cell_data['removed_base_segs']=set()

In [411]:
crest.add_endpoint_annotation_layers(['soma'],link=True) # spine_inputs
print(crest.get_ctype('manual'))

grc-d


In [8]:
base_segments_net = crest.get_base_segments_dict(Path(settings_dict['save_dir'])) 
base_segments_todo = crest.get_base_segments_dict(Path(settings_dict['save_dir']) / 'todo_presynaptic/lg_301787806')

In [16]:
print('overlap in main network:'); df = crest.check_duplicates(base_segments_net); display(df)
print('overlap in todo folder:'); df = crest.check_duplicates(base_segments_todo); display(df)

overlap in main network:


,self,dups,overlap-percent,number_seg_lap


overlap in todo folder:


,self,dups,overlap-percent,number_seg_lap
13,480314495,133489033,1.0,2.0


In [413]:
# SAVE YOUR WORK!

crest.save_cell_graph() # Default location is Path(settings_dict['save_dir'])

Saved cell 227330954 reconstruction locally at 2024-01-18 11.46.58


In [ ]:
cell_filepaths[cell_id].name

In [13]:
crest.get_ctype('manual')

'uk'

## Define Cell type

The cell type strings to use are:  
aff, grc-d, grc-s, sgx1, sgx2, sg1, sg2, mg1, mg2, lf, lg, uk, fov

You can also check the current cell type assigned by using ```crest.get_ctype("manual")```

In [ ]:
cell_type = 'grc-d' # Assign the cell type then run the code cell

crest.define_ctype(cell_type,"manual")

## Change Function Keybindings

speficy any keybindings that you want in ```keybindings_dict``` and pass that to the class module ```change_key_binding```. The keybinding mapping will be returned. 

In [ ]:
keybindings_dict = {
    "alt+mousedown0" : "add-or-remove-seg"
}

crest.change_key_binding(keybindings_dict)

## Check for overlap with other .json files in a directory

Use the ecrest module ```get_base_segments_dict``` to get a dictionary of {'cell_id' : 'base_segments'} for all .json files in a folder.  
Then use the ecrest module ```check_duplicates``` to get a dataframe of any instances of overlap between that cell and the .json files from the folder.

*Tips*:
- You do ***not*** need to run ```get_base_segments_dict``` every time. For each new crest instance you create, you can just skip to the ```check_duplicates``` step.
- You can create multiple dictionaries to check against... you can create a separate line of code for each. 

In [ ]:
base_segments_net = crest.get_base_segments_dict(Path(settings_dict['save_dir']))
base_segments_todo = crest.get_base_segments_dict(Path(settings_dict['save_dir']) / 'kp/392042360_grc-s_pre')

In [ ]:
print('overlap in main network:'); df = crest.check_duplicates(base_segments_net); display(df)
print('overlap in todo folder:'); df = crest.check_duplicates(base_segments_todo); display(df)

## Add/Remove Annotation layers

Because of how CREST saves the .json state, annotation layers need to be added/removed programatically rather than via the neuroglancer viewer directly.

Comment/uncomment the following two module implementations as needed.

In [ ]:
crest.add_endpoint_annotation_layers(['soma'])

# crest.del_endpoint_annotation_layers(['soma'])

## SAVE YOUR WORK!

In [ ]:
crest.save_cell_graph() # Default location is Path(settings_dict['save_dir'])

# Other

## search for synapse by segment annotated

In [363]:
directory_path = Path(settings_dict['save_dir']) #/ 'kp/pfs'#/ 'volume-subsample-all/in_progress'  ## specify the directory path
cell_filepaths = get_cell_filepaths(directory_path) # gets filepaths for all cells in a directory
# cell_filepaths = cell_filepaths_mainnet

In [367]:
         
syn_type = 'post-synaptic' #'spine_inputs' #
segs_to_find = ['564132646','565215897']
cells_todo = list(cell_filepaths.keys())

In [368]:
vx_sizes = [16, 16, 30]

for c_id in cells_todo:
    crest = ecrest(settings_dict,filepath= cell_filepaths[c_id], launch_viewer=False)
    for syn_ in crest.cell_data['end_points'][syn_type]:
        try:
            if syn_[3] in segs_to_find:
                print(f'in cell {c_id}, segment {syn_[3]} is at synapse location {array([int(syn_[i]/vx_sizes[i]) for i in range(3)])}')

        except IndexError as msg:
            print(msg, f'for cell {c_id} synapse at {array([int(syn_[i]/vx_sizes[i]) for i in range(3)])} has no segment id')

in cell 473631846, segment 565215897 is at synapse location [21129 18825  3239]
in cell 473631846, segment 564132646 is at synapse location [22832 17853  3135]


## cell types for all files in directory

In [658]:
nodefiles = get_cell_filepaths(Path(settings_dict['save_dir'])) 

In [659]:
cell_type = {}
not_typed = []
for x,f in nodefiles.items():
    cell = ecrest(settings_dict,filepath = f,launch_viewer=False)
    cell_type[x] = cell.get_ctype('manual') 
    if (cell.get_ctype('manual') == []) | (cell.get_ctype('manual') == ''):
        cell_type[x]=''
        not_typed.append(x)# print(f'cell {x} is not cell-typed in json')
        
print('the following cells are not typed in the main network')
print(not_typed)        
        

the following cells are not typed in the main network
[]


## if get assertion error and won't save

In [ ]:
seglist = crest.cell_data['base_segments']['unknown']

anchor = crest.cell_data['metadata']['main_seg']['base']

crest2 = ecrest(settings_dict,segment_id = anchor, segment_list = seglist, launch_viewer=True)

# SAVE YOUR WORK!

crest2.cell_data['end_points']=crest.cell_data['end_points']


crest2.load_annotation_layer_points()

crest2.save_cell_graph()

## eCREST from json segments

### neuroglancer file

In [23]:
neuroglancer_path = Path(settings_dict['save_dir']) / 'Nate_neuroglancer/synaptic_labeling/finished'
# neuroglancer_path = '/Users/kperks/Documents/gdrive/.shortcut-targets-by-id/16q1BuOMfD2ta0Cwq8CjMlRe4rDvbuWC5/ELL_connectome/CREST_reconstructions/mg-network/Nate_neuroglancer/synaptic_labeling/finished'
filename = '40448913_nbs.json'
neuroglancer_path = Path(neuroglancer_path) / filename

with open(Path(neuroglancer_path), 'r') as myfile: # 'p' is the dirpath and 'f' is the filename from the created 'd' dictionary
    neuroglancer_data = json.load(myfile)

In [24]:
segmentation_layer = next((item for item in neuroglancer_data['layers'] if item["source"] == 'brainmaps://10393113184:ell:roi450um_seg32fb16fb_220930'), None)
base_segment_list_ng = set(segmentation_layer['segments'])

In [25]:
base_segment_list_ng = list(set([bs for bs in base_segment_list_ng if '!' not in bs]))# -set(['283547027']))

### initiate cell if not already a crest file

In [ ]:
cell_id = filename.split('_')[0]
crest = ecrest(settings_dict, segment_id = segment_id, segment_list = base_segment_list_ng, launch_viewer=True)

### compare base segments with ecrest file

In [26]:
directory_path = Path(settings_dict['save_dir']) #/ 'todo_presynaptic/mg1_299496636' #/ 'kp/pfs'#/ 'volume-subsample-all/in_progress'  ## specify the directory path
cell_filepaths = get_cell_filepaths(directory_path) # gets filepaths for all cells in a directory
# cell_filepaths = cell_filepaths_mainnet

In [27]:
cell_id = filename.split('_')[0]
# crest = ecrest(settings_dict, segment_id = segment_id, segment_list = base_segment_list_ng, launch_viewer=True)

crest = ecrest(settings_dict, filepath= cell_filepaths[cell_id], launch_viewer=True)

updating viewer status message: Current Base Segment Counts: unknown: 3173, axon: 139, basal dendrite: 156, apical dendrite: 1446, dendrite: 0, multiple: 419


In [30]:
set(base_segment_list_ng) - set([a for _,b in crest.cell_data['base_segments'].items() for a in b])

set()

### Add points from annotation layers

In [28]:
[item['name'] for item in neuroglancer_data['layers'] if item['type']=='annotation']

['uncertain',
 'post-synaptic',
 'soma',
 'natural end',
 'exit volume',
 'pre-synaptic',
 'Base Segment Merger']

In [29]:
neuroglancer_layer_name = ['post-synaptic','pre-synaptic'] # [item['name'] for item in neuroglancer_data['layers'] if item['type']=='annotation']# ,
crest_layer_name = neuroglancer_layer_name #['post-synaptic','pre-synaptic']#,

for nl_, cl_ in zip(neuroglancer_layer_name, crest_layer_name):
    # get the 'layers' dictionary that has that name
    neuroglancer_layer = next((item for item in neuroglancer_data['layers'] if item["name"] == nl_), None)

    if neuroglancer_layer != None:
        if cl_ in crest.point_types:
            # add annotation layer
            crest.import_annotations(neuroglancer_data, [nl_], [cl_])
            print(f"Imported - {nl_} - layer from neuroglancer annotations tabs for cell {crest.cell_data['metadata']['main_seg']['base']} as - {cl_} -.")
        else: 
            msg = f"CREST layer name - {cl_} - incorrect for cell {crest.cell_data['metadata']['main_seg']['base']} in conversion_json"
            print(msg)

    else:
        msg = f"no layer by the name - {nl_} - in neuroglancer json for cell {crest.cell_data['metadata']['main_seg']['base']}"
        print(msg)

crest.load_annotation_layer_points()

Imported - post-synaptic - layer from neuroglancer annotations tabs for cell 40448913 as - post-synaptic -.
Imported - pre-synaptic - layer from neuroglancer annotations tabs for cell 40448913 as - pre-synaptic -.


### define type and save

In [983]:
cell_type = 'pf' # Assign the cell type then run the code cell

crest.define_ctype(cell_type,"manual")

In [32]:
# SAVE YOUR WORK!

crest.save_cell_graph() # Default location is Path(settings_dict['save_dir'])

Saved cell 40448913 reconstruction locally at 2024-01-16 09.14.02


In [ ]:
# 11100*16

In [ ]:
crest.cell_data['end_points']['uncertain'] = [r for r in crest.cell_data['end_points']['uncertain'] if r[1] < (11100*16)]

In [ ]:
crest.load_annotation_layer_points()

In [ ]:
# crest.add_endpoint_annotation_layers(['soma'])

In [ ]:
base_segments_net = crest.get_base_segments_dict(Path(settings_dict['save_dir']))
base_segments_todo = crest.get_base_segments_dict(Path(settings_dict['save_dir']) / 'todo_presynaptic/lf_393325331')

In [ ]:
print('overlap in main network:'); df = crest.check_duplicates(base_segments_net); display(df)
print('overlap in todo folder:'); df = crest.check_duplicates(base_segments_todo); display(df)

In [ ]:
cell_type = 'sgx1' # Assign the cell type then run the code cell

crest.define_ctype(cell_type,"manual")

In [1032]:
# SAVE YOUR WORK!

crest.save_cell_graph() # Default location is Path(settings_dict['save_dir'])

Saved cell 644824766 reconstruction locally at 2024-01-05 16.16.58


In [ ]:
# cell_filepaths[cell_id].name

In [ ]:
# crest.save_cell_graph(directory_path = cell_filepaths[cell_id].parent, file_name = cell_filepaths[cell_id].name)

In [ ]:
# crest.get_ctype('manual')